# Prediction of teams that will reach the Playoffs

## Data import

In [ ]:
import pandas as pd
import csv

players = pd.read_csv('basketballPlayoffs/players.csv', delimiter=",")
coaches = pd.read_csv('basketballPlayoffs/coaches.csv', delimiter=",")
teams = pd.read_csv('basketballPlayoffs/teams.csv', delimiter=",")
players_teams = pd.read_csv('basketballPlayoffs/players_teams.csv', delimiter=",")
teams_post = pd.read_csv('basketballPlayoffs/teams_post.csv', delimiter=",")
series_post = pd.read_csv('basketballPlayoffs/series_post.csv', delimiter=",")
awards_players = pd.read_csv('basketballPlayoffs/awards_players.csv', delimiter=",")
awards_coaches = pd.read_csv('basketballPlayoffs/awards_coaches.csv', delimiter=",")

print(players.head())
print(coaches)
print(teams)
print(players_teams)
print(teams_post)
print(series_post)
print(awards_players)
print(awards_coaches)


players.isnull().sum()
#players.describe()

## Data cleaning

Selecting relevant features and deleting unnecessary rows.

### Players dataset

In [ ]:
# to delete:  deathDate, collegeOther, , firstseason, lastseason linhas com pos=""
# show all players that have Pos: nullpd.set_option('display.max_rows', None)
print(players.isna().sum())
players = players.drop(columns=['collegeOther', 'deathDate', 'firstseason', 'lastseason'])
players = players[players['pos'] != ""]
print(players.head())

### Coaches dataset

In [ ]:
# to delete: lgID, post_wins, post_losses
print(coaches.isna().sum())
coaches = coaches.drop(columns=['lgID', 'post_wins', 'post_losses'])
print(coaches.head())

### Teams dataset

In [ ]:
# to delete: lgID, divID, seeded, tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB, rank, firstRound, semis, finals
print(teams.isna().sum())
   
teams = teams.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
teams = teams.drop(columns=['rank', 'firstRound', 'semis', 'finals'])
teams = teams.drop(columns=['min', 'o_oreb', 'o_dreb', 'd_oreb', 'd_dreb', 'arena', 'name'])

print(teams.head())

### Players Teams dataset

In [ ]:
# to delete: lgID 
# "PostGP","PostGS","PostMinutes","PostPoints","PostoRebounds","PostdRebounds","PostRebounds","PostAssists","PostSteals",
# "PostBlocks","PostTurnovers","PostPF","PostfgAttempted","PostfgMade","PostftAttempted","PostftMade","PostthreeAttempted","PostthreeMade","PostDQ"

#print(players_teams.isna().sum())
players_teams = players_teams.drop(columns=['lgID', "PostGP","PostGS","PostMinutes","PostPoints","PostoRebounds","PostdRebounds","PostRebounds","PostAssists","PostSteals","PostBlocks","PostTurnovers","PostPF","PostfgAttempted","PostfgMade","PostftAttempted","PostftMade","PostthreeAttempted","PostthreeMade","PostDQ"])
print(players_teams.head())

### Awards Players dataset

In [ ]:
# to delete: lgID
awards_players = awards_players.drop(columns=['lgID'])
print(awards_players.head())

### Awards Coaches dataset

In [ ]:
# to delete: lgID
#print(awards_players.isna().sum()) 
awards_coaches = awards_coaches.drop(columns=['lgID'])
print(awards_coaches.head())

## Feature Selection

### Player Rating


**Positions - Relevant Stats**

**Point Guard**
Assists(AST), Points(PTS), Assist-to-Turnover Ratio(AST/TO), Steals(STL), Three-Point Percentage(3P%), Free Throw Percentage(FT%), Minutes Played(MIN), Defensive Rating and Defensive Contribution(USG%), Clutch Performance.

**Shooting Guard**
Points per Game(PPG), Field Goal Percentage(FG%), Three-Point Percentage(3P%), Free Throw Percentage(FT%), Assists(AST), Rebounds(REB), Steals(STL), Blocks(BLK), Usage Rate(USG%), Effective Field Goal Percentage(eFG%)

**Small Forward**
Points per Game(PPG), Field Goal Percentage(FG%), Three-Point Percentage(3P%), Rebounds(REB), Assists(AST), Steals(STL), Blocks(BLK), Usage Rate(USG%), Player Efficiency Rating(PER)

**Power Forward**
Points per Game(PPG), Field Goal Percentage(FG%), Rebounds(REB), Defensive Rebounds(DREB), Blocks(BLK) Steals(STL), Usage Rate(USG%), Free Throw Percentage(FT%), Player Efficiency Rating(PER)

**Center**
Points per Game(PPG), Field Goal Percentage(FG%), Rebounds(REB), Defensive Rebounds(DREB), Blocks(BLK), Steals(STL), Usage Rate(USG%), Free Throw Percentage(FT%), Player Efficiency Rating(PER)


**Features in Dataset**

- GP (Games Played)
- GS (Games Started)
- minutes (Minutes Played)
- points (Points Scored)
- oRebounds (ofensive rebounds)
- dRebounds (defensive rebounds)
- rebounds (oRebounds + dRebounds)
- assists (assists)
- steals (steals)
- blocks (blocks)
- turnovers (turnovers)
- Personal Fouls (fouls)
- fgAttempted (field goals attempted)
- fgMade (field goals made)
- ftAttempted (free throws attempted)
- ftMade (free throws made)
- threeAttempted (three points attempted)
- threeMade (three points made)
- dq (times fouled out)

**Features to create**

- Points per Game (PPG): points/GP
- Field Goal Percentage(FG%): fgMade/fgAttempted
- Free Throw Percentage (FT%): ftMade/ftAttempted
- Three Points Percentage (3P%): threeAttempted/threeMade
- Effective Field Goal Percentage(eFG%): (fgMade + 0.5 * threeMade) / fgAttempted

https://captaincalculator.com/sports/basketball/efficiency/

In [ ]:
# highest and lowest value per feature to calculate rating (0-100)
pointsPerGameMax = 0
pointsPerGameMin = 999

assistPerGameMax = 0
assistPerGameMin = 999

reboundsPerGameMax = 0
reboundsPerGameMin = 999

defensiveReboundsPerGameMax = 0
defensiveReboundsPerGameMin = 999

stealsPerGameMax = 0
stealsPerGameMin = 999

blocksPerGameMax = 0
blocksPerGameMin = 999

turnoversPerGameMax = 0
turnoversPerGameMin = 999

assistToTurnoverRatioMax = 0
assistToTurnoverRatioMin = 999

minutesMax = 0
minutesMin = 999

# percentage
startsMax = 0
startsMin = 1

threePointPercentageMax = 0
threePointPercentageMin = 1

freeThrowPercentageMax = 0
freeThrowPercentageMin = 1

fieldGoalPercentageMax = 0
fieldGoalPercentageMin = 1

for index, row in players_teams.iterrows(): 
    if row['GP'] == 0:
        pointsPerGameMin = 0
    else: 
        if ( row['points'] / row['GP'] ) > pointsPerGameMax:
            pointsPerGameMax = ( row['points'] / row['GP'] )
        if ( row['points'] / row['GP'] ) < pointsPerGameMin:
            pointsPerGameMin = ( row['points'] / row['GP'] )
    
    if row['GP'] == 0:
        assistPerGameMin = 0
    else: 
        if ( row['assists'] / row['GP'] ) > assistPerGameMax:
            assistPerGameMax = ( row['assists'] / row['GP'] )
        if ( row['assists'] / row['GP'] ) < assistPerGameMin:
            assistPerGameMin = ( row['assists'] / row['GP'] )

    if row['GP'] == 0:
        reboundsPerGameMin = 0
    else: 
        if ( row['rebounds'] / row['GP'] ) > reboundsPerGameMax:
            reboundsPerGameMax = ( row['rebounds'] / row['GP'] )
        if ( row['rebounds'] / row['GP'] ) < reboundsPerGameMin:
            reboundsPerGameMin = ( row['rebounds'] / row['GP'] )

    if row['GP'] == 0:
        defensiveReboundsPerGameMin = 0
    else: 
        if ( row['dRebounds'] / row['GP'] ) > defensiveReboundsPerGameMax:
            defensiveReboundsPerGameMax = ( row['dRebounds'] / row['GP'] )
        if ( row['dRebounds'] / row['GP'] ) < defensiveReboundsPerGameMin:
            defensiveReboundsPerGameMin = ( row['dRebounds'] / row['GP'] )

    if row['GP'] == 0:
        stealsPerGameMin = 0
    else: 
        if ( row['steals'] / row['GP'] ) > stealsPerGameMax:
            stealsPerGameMax = ( row['steals'] / row['GP'] )
        if ( row['steals'] / row['GP'] ) < stealsPerGameMin:
            stealsPerGameMin = ( row['steals'] / row['GP'] )

    if row['GP'] == 0:
        blocksPerGameMin = 0
    else: 
        if ( row['blocks'] / row['GP'] ) > blocksPerGameMax:
            blocksPerGameMax = ( row['blocks'] / row['GP'] )
        if ( row['blocks'] / row['GP'] ) < blocksPerGameMin:
            blocksPerGameMin = ( row['blocks'] / row['GP'] )

    if row['GP'] == 0:
        turnoversPerGameMin = 0
    else: 
        if ( row['turnovers'] / row['GP'] ) > turnoversPerGameMax:
            turnoversPerGameMax = ( row['turnovers'] / row['GP'] )
        if ( row['turnovers'] / row['GP'] ) < turnoversPerGameMin:
            turnoversPerGameMin = ( row['turnovers'] / row['GP'] )

    if row['turnovers'] == 0 and row['assists'] == 0:
        assistToTurnoverRatioMin = 0
    elif row['turnovers'] == 0 and row['assists'] > 0:
        if(assistToTurnoverRatioMax < row['assists']): 
            assistToTurnoverRatioMax = row['assists']
    else: 
        if ( row['assists'] / row['turnovers'] ) > assistToTurnoverRatioMax:
            assistToTurnoverRatioMax = ( row['assists'] / row['turnovers'] )
        if ( row['assists'] / row['turnovers'] ) < assistToTurnoverRatioMin:
            assistToTurnoverRatioMin = ( row['assists'] / row['turnovers'] )

    if ( row['minutes'] ) > minutesMax:
        minutesMax = ( row['minutes'] )
    if ( row['minutes'] ) < minutesMin:
        minutesMin = ( row['minutes'] )

    if row['GP'] == 0:
        startsMin = 0
    else: 
        if ( row['GS'] / row['GP'] ) > startsMax:
            startsMax = ( row['GS'] / row['GP'] )
        if ( row['GS'] / row['GP'] ) < startsMin:
            startsMin = ( row['GS'] / row['GP'] )
    
    if row['threeAttempted'] == 0:
        threePointPercentageMin = 0
    else: 
        if ( row['threeMade'] / row['threeAttempted'] ) > threePointPercentageMax:
            threePointPercentageMax = ( row['threeMade'] / row['threeAttempted'] )
        if ( row['threeMade'] / row['threeAttempted'] ) < threePointPercentageMin:
            threePointPercentageMin = ( row['threeMade'] / row['threeAttempted'] )

    if row['ftAttempted'] == 0:
        freeThrowPercentageMin = 0
    else: 
        if ( row['ftMade'] / row['ftAttempted'] ) > freeThrowPercentageMax:
            freeThrowPercentageMax = ( row['ftMade'] / row['ftAttempted'] )
        if ( row['ftMade'] / row['ftAttempted'] ) < freeThrowPercentageMin:
            freeThrowPercentageMin = ( row['ftMade'] / row['ftAttempted'] )

    if row['fgAttempted'] == 0:
        fieldGoalPercentageMin = 0
    else: 
        if ( row['fgMade'] / row['fgAttempted'] ) > fieldGoalPercentageMax:
            fieldGoalPercentageMax = ( row['fgMade'] / row['fgAttempted'] )
        if ( row['fgMade'] / row['fgAttempted'] ) < fieldGoalPercentageMin:
            fieldGoalPercentageMin = ( row['fgMade'] / row['fgAttempted'] )

print("pointsPerGameMax: ", pointsPerGameMax)
print("pointsPerGameMin: ", pointsPerGameMin)
print("assistPerGameMax: ", assistPerGameMax)
print("assistPerGameMin: ", assistPerGameMin)
print("reboundsPerGameMax: ", reboundsPerGameMax)
print("reboundsPerGameMin: ", reboundsPerGameMin)
print("defensiveReboundsPerGameMax: ", defensiveReboundsPerGameMax)
print("defensiveReboundsPerGameMin: ", defensiveReboundsPerGameMin)
print("stealsPerGameMax: ", stealsPerGameMax)
print("stealsPerGameMin: ", stealsPerGameMin)
print("blocksPerGameMax: ", blocksPerGameMax)
print("blocksPerGameMin: ", blocksPerGameMin)
print("turnoversPerGameMax: ", turnoversPerGameMax)
print("turnoversPerGameMin: ", turnoversPerGameMin)
print("assistToTurnoverRatioMax: ", assistToTurnoverRatioMax)
print("assistToTurnoverRatioMin: ", assistToTurnoverRatioMin)
print("minutesMax: ", minutesMax)
print("minutesMin: ", minutesMin)
print("startsMax: ", startsMax)
print("startsMin: ", startsMin)
print("threePointPercentageMax: ", threePointPercentageMax)
print("threePointPercentageMin: ", threePointPercentageMin)
print("freeThrowPercentageMax: ", freeThrowPercentageMax)
print("freeThrowPercentageMin: ", freeThrowPercentageMin)
print("fieldGoalPercentageMax: ", fieldGoalPercentageMax)
print("fieldGoalPercentageMin: ", fieldGoalPercentageMin)

#### **Forward**

Points per Game(PPG), Field Goal Percentage(FG%), Free Throw Percentage(FT%), Three-Point Percentage(3P%), Rebounds(REB), Assists(AST), Steals(STL), Blocks(BLK), Usage Rate(USG%), Player Efficiency Rating(PER)

Features we do have in dataset:
- points/GP, fgMade/fgAttempted, ftMade/ftAttempted, threeMade/threeAttempted, rebounds, assists, steals, blocks, minutes, GS/GP

In [ ]:
def forward_power_ranking(player): 

    if(player['fgAttempted'] == 0):
        fieldGoalPercentage = 0
    else:
        fieldGoalPercentage = (player['fgMade'] / player['fgAttempted']) * 100

    if(player['threeAttempted'] == 0):
        threePointPercentage = 0
    else:
        threePointPercentage = (player['threeMade'] / player['threeAttempted']) * 100

    if(player['ftAttempted'] == 0):
        freeThrowPercentage = 0
    else:
        freeThrowPercentage = (player['ftMade'] / player['ftAttempted']) * 100

    if(player['GP'] == 0): 
        pointsPerGame = 0
        reboundsPerGame = 0
        assistsPerGame = 0
        stealsPerGame = 0
        blocksPerGame = 0
        starts = 0
    else:
        pointsPerGame = (((player['points'] / player['GP']) - pointsPerGameMin) / (pointsPerGameMax - pointsPerGameMin)) * 100
        reboundsPerGame = (((player['rebounds'] / player['GP']) - reboundsPerGameMin) / (reboundsPerGameMax - reboundsPerGameMin)) * 100
        assistsPerGame = (((player['assists'] / player['GP']) - assistPerGameMin) / (assistPerGameMax - assistPerGameMin)) * 100
        stealsPerGame = (((player['steals'] / player['GP']) - stealsPerGameMin) / (stealsPerGameMax - stealsPerGameMin)) * 100
        blocksPerGame = (((player['blocks'] / player['GP']) - blocksPerGameMin) / (blocksPerGameMax - blocksPerGameMin)) * 100
        starts = (player['GS'] / player['GP']) * 100

    minutes = (player['minutes'] / minutesMax) * 100

    relevantFeatures = [pointsPerGame, fieldGoalPercentage, freeThrowPercentage, threePointPercentage, reboundsPerGame, assistsPerGame, stealsPerGame, blocksPerGame, minutes, starts]

    powerRankingAcummulator = 0
    numberOfFeatures = len(relevantFeatures)
    minutesWeight = ((100 / numberOfFeatures) * 2.0) / 100
    averageFeatureWeight = (1 - minutesWeight) / (numberOfFeatures - 1)

    for feature in relevantFeatures: 
        if feature == minutes: 
            powerRankingAcummulator += feature * minutesWeight

        else:
            powerRankingAcummulator += feature * averageFeatureWeight
            

    return powerRankingAcummulator

#### **Guard**

Points per Game(PPG), Field Goal Percentage(FG%), Three-Point Percentage(3P%), Free Throw Percentage(FT%), Assist-to-Turnover Ratio(AST/TO), Assists(AST), Rebounds(REB), Steals(STL), Blocks(BLK), Usage Rate(USG%), Effective Field Goal Percentage(eFG%)

Features we do have in dataset: 
- points/GP, fgMade/fgAttempted, threeMade/threeAttempted, ftMade/ftAttempted, assists/turnovers, assists, rebounds, steals, blocks, minutes, GS/GP

In [ ]:
def guard_power_ranking(player): 
    
    if(player['fgAttempted'] == 0):
        fieldGoalPercentage = 0
    else:
        fieldGoalPercentage = (player['fgMade'] / player['fgAttempted']) * 100

    if(player['threeAttempted'] == 0):
        threePointPercentage = 0
    else:
        threePointPercentage = (player['threeMade'] / player['threeAttempted']) * 100

    if(player['ftAttempted'] == 0):
        freeThrowPercentage = 0
    else:
        freeThrowPercentage = (player['ftMade'] / player['ftAttempted']) * 100

    if(player['turnovers'] == 0):
        assistToTurnoverRatio = player['assists']
    else:
        assistToTurnoverRatio = (((player['assists'] / player['turnovers']) - assistToTurnoverRatioMin) / (assistToTurnoverRatioMax - assistToTurnoverRatioMin)) * 100

    if(player['GP'] == 0):
        pointsPerGame = 0
        assistsPerGame = 0
        reboundsPerGame = 0
        stealsPerGame = 0
        blocksPerGame = 0
        starts = 0
    else:
        pointsPerGame = (((player['points'] / player['GP']) - pointsPerGameMin) / (pointsPerGameMax - pointsPerGameMin)) * 100
        assistsPerGame = (((player['assists'] / player['GP']) - assistPerGameMin) / (assistPerGameMax - assistPerGameMin)) * 100
        reboundsPerGame = (((player['rebounds'] / player['GP']) - reboundsPerGameMin) / (reboundsPerGameMax - reboundsPerGameMin)) * 100
        stealsPerGame = (((player['steals'] / player['GP']) - stealsPerGameMin) / (stealsPerGameMax - stealsPerGameMin)) * 100
        blocksPerGame = (((player['blocks'] / player['GP']) - blocksPerGameMin) / (blocksPerGameMax - blocksPerGameMin)) * 100
        starts = (player['GS'] / player['GP']) * 100

    minutes = (player['minutes'] / minutesMax) * 100

    relevantFeatures = [pointsPerGame, fieldGoalPercentage, threePointPercentage, freeThrowPercentage, assistToTurnoverRatio, assistsPerGame, reboundsPerGame, stealsPerGame, blocksPerGame, minutes, starts]

    powerRankingAcummulator = 0
    numberOfFeatures = len(relevantFeatures)
    minutesWeight = ((100 / numberOfFeatures) * 2.0) / 100
    averageFeatureWeight = (1 - minutesWeight) / (numberOfFeatures - 1)

    for feature in relevantFeatures: 
        if feature == minutes: 
            powerRankingAcummulator += feature * minutesWeight

        else:
            powerRankingAcummulator += feature * averageFeatureWeight
            

    return powerRankingAcummulator

#### **Center**

Points per Game(PPG), Field Goal Percentage(FG%), Rebounds(REB), Defensive Rebounds(DREB), Blocks(BLK), Steals(STL), Usage Rate(USG%), Free Throw Percentage(FT%), Player Efficiency Rating(PER)

Features we do have in dataset: 
-  points/GP, fgMade/fgAttempted, rebounds, dRebounds, blocks, steals, minutes, GS/GP, ftMade/ftAttempted

In [ ]:
def center_power_ranking(player): 

    if(player['fgAttempted'] == 0):
        fieldGoalPercentage = 0
    else:
        fieldGoalPercentage = (player['fgMade'] / player['fgAttempted']) * 100

    if(player['ftAttempted'] == 0):
        freeThrowPercentage = 0
    else:
        freeThrowPercentage = (player['ftMade'] / player['ftAttempted']) * 100

    if(player['GP'] == 0): 
        pointsPerGame = 0
        reboundsPerGame = 0
        defensiveReboundsPerGame = 0
        blocksPerGame = 0
        stealsPerGame = 0
        starts = 0
    else:
        pointsPerGame = (((player['points'] / player['GP']) - pointsPerGameMin) / (pointsPerGameMax - pointsPerGameMin)) * 100
        reboundsPerGame = (((player['rebounds'] / player['GP']) - reboundsPerGameMin) / (reboundsPerGameMax - reboundsPerGameMin)) * 100
        defensiveReboundsPerGame = (((player['dRebounds'] / player['GP']) - defensiveReboundsPerGameMin) / (defensiveReboundsPerGameMax - defensiveReboundsPerGameMin)) * 100
        blocksPerGame = (((player['blocks'] / player['GP']) - blocksPerGameMin) / (blocksPerGameMax - blocksPerGameMin)) * 100
        stealsPerGame = (((player['steals'] / player['GP']) - stealsPerGameMin) / (stealsPerGameMax - stealsPerGameMin)) * 100
        starts = (player['GS'] / player['GP']) * 100

    minutes = (player['minutes'] / minutesMax) * 100

    relevantFeatures = [pointsPerGame, fieldGoalPercentage, freeThrowPercentage, reboundsPerGame, defensiveReboundsPerGame, blocksPerGame, stealsPerGame, minutes, starts]

    powerRankingAcummulator = 0
    numberOfFeatures = len(relevantFeatures)
    minutesWeight = ((100 / numberOfFeatures) * 2.0) / 100
    averageFeatureWeight = (1 - minutesWeight) / (numberOfFeatures - 1)

    for feature in relevantFeatures: 
        if feature == minutes: 
            powerRankingAcummulator += feature * minutesWeight

        else:
            powerRankingAcummulator += feature * averageFeatureWeight
            

    return powerRankingAcummulator

#### Calculate a Player Rating

In [ ]:
merge_players_players_teams = pd.merge(players, players_teams, how='inner', on='playerID')

# 'G' , 'F', 'C'

for index, player in merge_players_players_teams.iterrows(): 
    if 'G' in player['pos']:
        merge_players_players_teams.at[index, 'powerRanking'] = guard_power_ranking(player)
    elif 'F' in player['pos']:
        merge_players_players_teams.at[index, 'powerRanking'] = forward_power_ranking(player)
    elif 'C' in player['pos']:
        merge_players_players_teams.at[index, 'powerRanking'] = center_power_ranking(player)

In [ ]:
# GP,GS,minutes,points,oRebounds,dRebounds,rebounds,assists,steals,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade
merge_players_players_teams = merge_players_players_teams.drop(columns=['GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade'])

merge_players_players_teams.to_csv('players_teams.csv', index=False)

print(players_teams.head)

### Team Rating

Minutes that teams played does not look like a relevant feature, as the difference between the biggest number of minutes played and the least is not that big. 

In [ ]:
# to remove
#'lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'rank', 'firstRound', 'semis', 'finals'
# 'min', 'o_oreb', 'o_dreb', 'd_oreb', 'd_dreb', 'arena', 'name'
#maxMinutes = teams['min'].max()
#minMinutes = teams['min'].min()

#print(maxMinutes)
#print(minMinutes)

**Offensive stats**
- field goal percentage (o_fgm/o_fga)
- free throw percentage (o_ftm/o_fta)
- three point percentage (o_3pm/o_3pa)
- rebounds per game (o_reb / GP)
- assists per game (o_asts / GP)
- personal fouls per game (o_pf / GP) 
- steals per game (o_stl / GP)
- turnovers per game (o_to / GP)
- blocks per game (o_blk / GP)
- points per game (o_pts / GP)

**Defensive stats**
- field goal percentage (d_fgm/d_fga)
- free throw percentage (d_ftm/d_fta)
- three point percentage (d_3pm/d_3pa)
- rebounds per game (d_reb / GP)
- assists per game (d_asts / GP)
- personal fouls per game (d_pf / GP) 
- steals per game (d_stl / GP)
- turnovers per game (d_to / GP)
- blocks per game (d_blk / GP)
- points per game (d_pts / GP)


**Global stats**
- game winning percentage (won / GP)
- games won (won)
- games lost (lost)
- games played (GP)
- home win percentage ((homeW) / (homeW + homeL))
- home loss percentage ((homeL) / (homeW + homeL))
- away win percentage ((awayW) / (awayW + awayL))
- away loss percentage ((awayL) / (awayW + awayL))
- conference win percentage ((confW) / (confW + confL))
- conference loss percentage ((confL) / (confW + confL))
- attendence (attend) (might be relevant as we believe in the crowd influence, to keep it safe we are attributing low percentage)

"It turns out that performing in front of a crowd can be a great distraction to fatigue. Focusing on the crowd instead of their pain or exhaustion has been demonstrated to be an effective strategy to help athletes when they are tiring and about to 'hit the wall'." ( https://blog.innerdrive.co.uk/sports/psychology-playing-behind-closed-doors, 22nd oct)

In [ ]:
# offensive stats
offensiveFieldGoalPercentageMax = 0
offensiveFieldGoalPercentageMin = 100

offensiveFreeThrowPercentageMax = 0
offensiveFreeThrowPercentageMin = 100

offensiveThreePointPercentageMax = 0
offensiveThreePointPercentageMin = 100

offensiveReboundsPerGameMax = 0
offensiveReboundsPerGameMin = 999

offensiveAssistsPerGameMax = 0
offensiveAssistsPerGameMin = 999

offensivePersonalFoulsPerGameMax = 0
offensivePersonalFoulsPerGameMin = 999

offensiveStealsPerGameMax = 0
offensiveStealsPerGameMin = 999

offensiveTurnoversPerGameMax = 0
offensiveTurnoversPerGameMin = 999

offensiveBlocksPerGameMax = 0
offensiveBlocksPerGameMin = 999

offensivePointsPerGameMax = 0
offensivePointsPerGameMin = 999

In [ ]:
# defensive stats
defensiveFieldGoalPercentageMax = 0
defensiveFieldGoalPercentageMin = 100

defensiveFreeThrowPercentageMax = 0
defensiveFreeThrowPercentageMin = 100

defensiveThreePointPercentageMax = 0
defensiveThreePointPercentageMin = 100

defensiveReboundsPerGameMax = 0
defensiveReboundsPerGameMin = 999

defensiveAssistsPerGameMax = 0
defensiveAssistsPerGameMin = 999

defensivePersonalFoulsPerGameMax = 0
defensivePersonalFoulsPerGameMin = 999

defensiveStealsPerGameMax = 0
defensiveStealsPerGameMin = 999

defensiveTurnoversPerGameMax = 0
defensiveTurnoversPerGameMin = 999

defensiveBlocksPerGameMax = 0
defensiveBlocksPerGameMin = 999

defensivePointsPerGameMax = 0
defensivePointsPerGameMin = 999

In [ ]:
# global stats
winPercentageMax = 0
winPercentageMin = 100

homeWinPercentageMax = 0
homeWinPercentageMin = 100

awayWinPercentageMax = 0
awayWinPercentageMin = 100

conferenceWinPercentageMax = 0
conferenceWinPercentageMin = 100

attendanceMax = 0
attendanceMin = 9999999

In [ ]:
for index, row in teams.iterrows():
    # offensive stats
    offensiveFieldGoalPercentage = (row['o_fgm'] / row['o_fga']) * 100

    if(offensiveFieldGoalPercentage > offensiveFieldGoalPercentageMax):
        offensiveFieldGoalPercentageMax = offensiveFieldGoalPercentage
    if(offensiveFieldGoalPercentage < offensiveFieldGoalPercentageMin):
        offensiveFieldGoalPercentageMin = offensiveFieldGoalPercentage

    offensiveFreeThrowPercentage = (row['o_ftm'] / row['o_fta']) * 100

    if(offensiveFreeThrowPercentage > offensiveFreeThrowPercentageMax):
        offensiveFreeThrowPercentageMax = offensiveFreeThrowPercentage
    if(offensiveFreeThrowPercentage < offensiveFreeThrowPercentageMin):
        offensiveFreeThrowPercentageMin = offensiveFreeThrowPercentage

    offensiveThreePointPercentage = (row['o_3pm'] / row['o_3pa']) * 100

    if(offensiveThreePointPercentage > offensiveThreePointPercentageMax):
        offensiveThreePointPercentageMax = offensiveThreePointPercentage
    if(offensiveThreePointPercentage < offensiveThreePointPercentageMin):
        offensiveThreePointPercentageMin = offensiveThreePointPercentage

    offensiveReboundsPerGame = row['o_reb'] / row['GP']

    if(offensiveReboundsPerGame > offensiveReboundsPerGameMax):
        offensiveReboundsPerGameMax = offensiveReboundsPerGame
    if(offensiveReboundsPerGame < offensiveReboundsPerGameMin):
        offensiveReboundsPerGameMin = offensiveReboundsPerGame

    offensiveAssistsPerGame = row['o_asts'] / row['GP']

    if(offensiveAssistsPerGame > offensiveAssistsPerGameMax):
        offensiveAssistsPerGameMax = offensiveAssistsPerGame
    if(offensiveAssistsPerGame < offensiveAssistsPerGameMin):
        offensiveAssistsPerGameMin = offensiveAssistsPerGame

    offensivePersonalFoulsPerGame = row['o_pf'] / row['GP']

    if(offensivePersonalFoulsPerGame > offensivePersonalFoulsPerGameMax):
        offensivePersonalFoulsPerGameMax = offensivePersonalFoulsPerGame
    if(offensivePersonalFoulsPerGame < offensivePersonalFoulsPerGameMin):
        offensivePersonalFoulsPerGameMin = offensivePersonalFoulsPerGame

    offensiveStealsPerGame = row['o_stl'] / row['GP']

    if(offensiveStealsPerGame > offensiveStealsPerGameMax):
        offensiveStealsPerGameMax = offensiveStealsPerGame
    if(offensiveStealsPerGame < offensiveStealsPerGameMin):
        offensiveStealsPerGameMin = offensiveStealsPerGame

    offensiveTurnoversPerGame = row['o_to'] / row['GP']

    if(offensiveTurnoversPerGame > offensiveTurnoversPerGameMax):
        offensiveTurnoversPerGameMax = offensiveTurnoversPerGame
    if(offensiveTurnoversPerGame < offensiveTurnoversPerGameMin):
        offensiveTurnoversPerGameMin = offensiveTurnoversPerGame

    offensiveBlocksPerGame = row['o_blk'] / row['GP']

    if(offensiveBlocksPerGame > offensiveBlocksPerGameMax):
        offensiveBlocksPerGameMax = offensiveBlocksPerGame
    if(offensiveBlocksPerGame < offensiveBlocksPerGameMin):
        offensiveBlocksPerGameMin = offensiveBlocksPerGame

    offensivePointsPerGame = row['o_pts'] / row['GP']

    if(offensivePointsPerGame > offensivePointsPerGameMax):
        offensivePointsPerGameMax = offensivePointsPerGame
    if(offensivePointsPerGame < offensivePointsPerGameMin):
        offensivePointsPerGameMin = offensivePointsPerGame

    # defensive stats
    defensiveFieldGoalPercentage = (row['d_fgm'] / row['d_fga']) * 100

    if(defensiveFieldGoalPercentage > defensiveFieldGoalPercentageMax):
        defensiveFieldGoalPercentageMax = defensiveFieldGoalPercentage
    if(defensiveFieldGoalPercentage < defensiveFieldGoalPercentageMin):
        defensiveFieldGoalPercentageMin = defensiveFieldGoalPercentage

    defensiveFreeThrowPercentage = (row['d_ftm'] / row['d_fta']) * 100

    if(defensiveFreeThrowPercentage > defensiveFreeThrowPercentageMax):
        defensiveFreeThrowPercentageMax = defensiveFreeThrowPercentage
    if(defensiveFreeThrowPercentage < defensiveFreeThrowPercentageMin):
        defensiveFreeThrowPercentageMin = defensiveFreeThrowPercentage

    defensiveThreePointPercentage = (row['d_3pm'] / row['d_3pa']) * 100

    if(defensiveThreePointPercentage > defensiveThreePointPercentageMax):
        defensiveThreePointPercentageMax = defensiveThreePointPercentage
    if(defensiveThreePointPercentage < defensiveThreePointPercentageMin):
        defensiveThreePointPercentageMin = defensiveThreePointPercentage

    defensiveReboundsPerGame = row['d_reb'] / row['GP']

    if(defensiveReboundsPerGame > defensiveReboundsPerGameMax):
        defensiveReboundsPerGameMax = defensiveReboundsPerGame
    if(defensiveReboundsPerGame < defensiveReboundsPerGameMin):
        defensiveReboundsPerGameMin = defensiveReboundsPerGame

    defensiveAssistsPerGame = row['d_asts'] / row['GP']

    if(defensiveAssistsPerGame > defensiveAssistsPerGameMax):
        defensiveAssistsPerGameMax = defensiveAssistsPerGame
    if(defensiveAssistsPerGame < defensiveAssistsPerGameMin):
        defensiveAssistsPerGameMin = defensiveAssistsPerGame

    defensivePersonalFoulsPerGame = row['d_pf'] / row['GP']

    if(defensivePersonalFoulsPerGame > defensivePersonalFoulsPerGameMax):
        defensivePersonalFoulsPerGameMax = defensivePersonalFoulsPerGame
    if(defensivePersonalFoulsPerGame < defensivePersonalFoulsPerGameMin):
        defensivePersonalFoulsPerGameMin = defensivePersonalFoulsPerGame

    defensiveStealsPerGame = row['d_stl'] / row['GP']

    if(defensiveStealsPerGame > defensiveStealsPerGameMax):
        defensiveStealsPerGameMax = defensiveStealsPerGame
    if(defensiveStealsPerGame < defensiveStealsPerGameMin):
        defensiveStealsPerGameMin = defensiveStealsPerGame

    defensiveTurnoversPerGame = row['d_to'] / row['GP']

    if(defensiveTurnoversPerGame > defensiveTurnoversPerGameMax):
        defensiveTurnoversPerGameMax = defensiveTurnoversPerGame
    if(defensiveTurnoversPerGame < defensiveTurnoversPerGameMin):
        defensiveTurnoversPerGameMin = defensiveTurnoversPerGame

    defensiveBlocksPerGame = row['d_blk'] / row['GP']

    if(defensiveBlocksPerGame > defensiveBlocksPerGameMax):
        defensiveBlocksPerGameMax = defensiveBlocksPerGame
    if(defensiveBlocksPerGame < defensiveBlocksPerGameMin):
        defensiveBlocksPerGameMin = defensiveBlocksPerGame

    defensivePointsPerGame = row['d_pts'] / row['GP']

    if(defensivePointsPerGame > defensivePointsPerGameMax):
        defensivePointsPerGameMax = defensivePointsPerGame
    if(defensivePointsPerGame < defensivePointsPerGameMin):
        defensivePointsPerGameMin = defensivePointsPerGame

    winPercentage = row['won'] / row['GP'] * 100

    if(winPercentage > winPercentageMax):
        winPercentageMax = winPercentage
    if(winPercentage < winPercentageMin):
        winPercentageMin = winPercentage

    homeWinPercentage = row['homeW'] / (row['homeW'] + row['homeL']) * 100

    if(homeWinPercentage > homeWinPercentageMax):
        homeWinPercentageMax = homeWinPercentage    
    if(homeWinPercentage < homeWinPercentageMin):
        homeWinPercentageMin = homeWinPercentage

    awayWinPercentage = row['awayW'] / (row['awayW'] + row['awayL']) * 100

    if(awayWinPercentage > awayWinPercentageMax):
        awayWinPercentageMax = awayWinPercentage
    if(awayWinPercentage < awayWinPercentageMin):
        awayWinPercentageMin = awayWinPercentage

    conferenceWinPercentage = row['confW'] / (row['confW'] + row['confL']) * 100

    if(conferenceWinPercentage > conferenceWinPercentageMax):
        conferenceWinPercentageMax = conferenceWinPercentage
    if(conferenceWinPercentage < conferenceWinPercentageMin):
        conferenceWinPercentageMin = conferenceWinPercentage

    if(row['attend'] > attendanceMax):
        attendanceMax = row['attend']
    if(row['attend'] < attendanceMin):
        attendanceMin = row['attend']

In [ ]:
print("offensiveFieldGoalPercentageMax: ", offensiveFieldGoalPercentageMax)
print("offensiveFieldGoalPercentageMin: ", offensiveFieldGoalPercentageMin)
print("offensiveFreeThrowPercentageMax: ", offensiveFreeThrowPercentageMax)
print("offensiveFreeThrowPercentageMin: ", offensiveFreeThrowPercentageMin)
print("offensiveThreePointPercentageMax: ", offensiveThreePointPercentageMax)
print("offensiveThreePointPercentageMin: ", offensiveThreePointPercentageMin)
print("offensiveReboundsPerGameMax: ", offensiveReboundsPerGameMax)
print("offensiveReboundsPerGameMin: ", offensiveReboundsPerGameMin)
print("offensiveAssistsPerGameMax: ", offensiveAssistsPerGameMax)
print("offensiveAssistsPerGameMin: ", offensiveAssistsPerGameMin)
print("offensivePersonalFoulsPerGameMax: ", offensivePersonalFoulsPerGameMax)
print("offensivePersonalFoulsPerGameMin: ", offensivePersonalFoulsPerGameMin)
print("offensiveStealsPerGameMax: ", offensiveStealsPerGameMax)
print("offensiveStealsPerGameMin: ", offensiveStealsPerGameMin)
print("offensiveTurnoversPerGameMax: ", offensiveTurnoversPerGameMax)
print("offensiveTurnoversPerGameMin: ", offensiveTurnoversPerGameMin)
print("offensiveBlocksPerGameMax: ", offensiveBlocksPerGameMax)
print("offensiveBlocksPerGameMin: ", offensiveBlocksPerGameMin)
print("offensivePointsPerGameMax: ", offensivePointsPerGameMax)
print("offensivePointsPerGameMin: ", offensivePointsPerGameMin)

In [ ]:
print("defensiveFieldGoalPercentageMax: ", defensiveFieldGoalPercentageMax)
print("defensiveFieldGoalPercentageMin: ", defensiveFieldGoalPercentageMin)
print("defensiveFreeThrowPercentageMax: ", defensiveFreeThrowPercentageMax)
print("defensiveFreeThrowPercentageMin: ", defensiveFreeThrowPercentageMin)
print("defensiveThreePointPercentageMax: ", defensiveThreePointPercentageMax)
print("defensiveThreePointPercentageMin: ", defensiveThreePointPercentageMin)
print("defensiveReboundsPerGameMax: ", defensiveReboundsPerGameMax)
print("defensiveReboundsPerGameMin: ", defensiveReboundsPerGameMin)
print("defensiveAssistsPerGameMax: ", defensiveAssistsPerGameMax)
print("defensiveAssistsPerGameMin: ", defensiveAssistsPerGameMin)
print("defensivePersonalFoulsPerGameMax: ", defensivePersonalFoulsPerGameMax)
print("defensivePersonalFoulsPerGameMin: ", defensivePersonalFoulsPerGameMin)
print("defensiveStealsPerGameMax: ", defensiveStealsPerGameMax)
print("defensiveStealsPerGameMin: ", defensiveStealsPerGameMin)
print("defensiveTurnoversPerGameMax: ", defensiveTurnoversPerGameMax)
print("defensiveTurnoversPerGameMin: ", defensiveTurnoversPerGameMin)
print("defensiveBlocksPerGameMax: ", defensiveBlocksPerGameMax)
print("defensiveBlocksPerGameMin: ", defensiveBlocksPerGameMin)
print("defensivePointsPerGameMax: ", defensivePointsPerGameMax)
print("defensivePointsPerGameMin: ", defensivePointsPerGameMin)

In [ ]:
# global stats
print("winPercentageMax: ", winPercentageMax)
print("winPercentageMin: ", winPercentageMin)
print("homeWinPercentageMax: ", homeWinPercentageMax)
print("homeWinPercentageMin: ", homeWinPercentageMin)
print("awayWinPercentageMax: ", awayWinPercentageMax)
print("awayWinPercentageMin: ", awayWinPercentageMin)
print("conferenceWinPercentageMax: ", conferenceWinPercentageMax)
print("conferenceWinPercentageMin: ", conferenceWinPercentageMin)
print("attendanceMax: ", attendanceMax)
print("attendanceMin: ", attendanceMin)

#### Calculate Team Rating

In [ ]:
def team_power_ranking(team): 

    # offensive stats
    offensiveFieldGoalPercentage = (team['o_fgm'] / team['o_fga']) * 100
    offensiveFieldGoalPercentage = (offensiveFieldGoalPercentage - offensiveFieldGoalPercentageMin) / (offensiveFieldGoalPercentageMax - offensiveFieldGoalPercentageMin) * 100

    offensiveFreeThrowPercentage = (team['o_ftm'] / team['o_fta']) * 100
    offensiveFreeThrowPercentage = (offensiveFreeThrowPercentage - offensiveFreeThrowPercentageMin) / (offensiveFreeThrowPercentageMax - offensiveFreeThrowPercentageMin) * 100

    offensiveThreePointPercentage = (team['o_3pm'] / team['o_3pa']) * 100
    offensiveThreePointPercentage = (offensiveThreePointPercentage - offensiveThreePointPercentageMin) / (offensiveThreePointPercentageMax - offensiveThreePointPercentageMin) * 100

    offensiveReboundsPerGame = team['o_reb'] / team['GP']
    offensiveReboundsPerGame = (offensiveReboundsPerGame - offensiveReboundsPerGameMin) / (offensiveReboundsPerGameMax - offensiveReboundsPerGameMin) * 100

    offensiveAssistsPerGame = team['o_asts'] / team['GP']
    offensiveAssistsPerGame = (offensiveAssistsPerGame - offensiveAssistsPerGameMin) / (offensiveAssistsPerGameMax - offensiveAssistsPerGameMin) * 100

    offensivePersonalFoulsPerGame = team['o_pf'] / team['GP']
    offensivePersonalFoulsPerGame = (offensivePersonalFoulsPerGame - offensivePersonalFoulsPerGameMin) / (offensivePersonalFoulsPerGameMax - offensivePersonalFoulsPerGameMin) * 100

    offensiveStealsPerGame = team['o_stl'] / team['GP']
    offensiveStealsPerGame = (offensiveStealsPerGame - offensiveStealsPerGameMin) / (offensiveStealsPerGameMax - offensiveStealsPerGameMin) * 100

    offensiveTurnoversPerGame = team['o_to'] / team['GP']
    offensiveTurnoversPerGame = (offensiveTurnoversPerGame - offensiveTurnoversPerGameMin) / (offensiveTurnoversPerGameMax - offensiveTurnoversPerGameMin) * 100

    offensiveBlocksPerGame = team['o_blk'] / team['GP']
    offensiveBlocksPerGame = (offensiveBlocksPerGame - offensiveBlocksPerGameMin) / (offensiveBlocksPerGameMax - offensiveBlocksPerGameMin) * 100

    offensivePointsPerGame = team['o_pts'] / team['GP']
    offensivePointsPerGame = (offensivePointsPerGame - offensivePointsPerGameMin) / (offensivePointsPerGameMax - offensivePointsPerGameMin) * 100

    # defensive stats
    defensiveFieldGoalPercentage = (team['d_fgm'] / team['d_fga']) * 100
    defensiveFieldGoalPercentage = (defensiveFieldGoalPercentage - defensiveFieldGoalPercentageMin) / (defensiveFieldGoalPercentageMax - defensiveFieldGoalPercentageMin) * 100

    defensiveFreeThrowPercentage = (team['d_ftm'] / team['d_fta']) * 100
    defensiveFreeThrowPercentage = (defensiveFreeThrowPercentage - defensiveFreeThrowPercentageMin) / (defensiveFreeThrowPercentageMax - defensiveFreeThrowPercentageMin) * 100

    defensiveThreePointPercentage = (team['d_3pm'] / team['d_3pa']) * 100
    defensiveThreePointPercentage = (defensiveThreePointPercentage - defensiveThreePointPercentageMin) / (defensiveThreePointPercentageMax - defensiveThreePointPercentageMin) * 100

    defensiveReboundsPerGame = team['d_reb'] / team['GP']
    defensiveReboundsPerGame = (defensiveReboundsPerGame - defensiveReboundsPerGameMin) / (defensiveReboundsPerGameMax - defensiveReboundsPerGameMin) * 100

    defensiveAssistsPerGame = team['d_asts'] / team['GP']
    defensiveAssistsPerGame = (defensiveAssistsPerGame - defensiveAssistsPerGameMin) / (defensiveAssistsPerGameMax - defensiveAssistsPerGameMin) * 100

    defensivePersonalFoulsPerGame = team['d_pf'] / team['GP']
    defensivePersonalFoulsPerGame = (defensivePersonalFoulsPerGame - defensivePersonalFoulsPerGameMin) / (defensivePersonalFoulsPerGameMax - defensivePersonalFoulsPerGameMin) * 100

    defensiveStealsPerGame = team['d_stl'] / team['GP']
    defensiveStealsPerGame = (defensiveStealsPerGame - defensiveStealsPerGameMin) / (defensiveStealsPerGameMax - defensiveStealsPerGameMin) * 100

    defensiveTurnoversPerGame = team['d_to'] / team['GP']
    defensiveTurnoversPerGame = (defensiveTurnoversPerGame - defensiveTurnoversPerGameMin) / (defensiveTurnoversPerGameMax - defensiveTurnoversPerGameMin) * 100

    defensiveBlocksPerGame = team['d_blk'] / team['GP']
    defensiveBlocksPerGame = (defensiveBlocksPerGame - defensiveBlocksPerGameMin) / (defensiveBlocksPerGameMax - defensiveBlocksPerGameMin) * 100

    defensivePointsPerGame = team['d_pts'] / team['GP']
    defensivePointsPerGame = (defensivePointsPerGame - defensivePointsPerGameMin) / (defensivePointsPerGameMax - defensivePointsPerGameMin) * 100

    # global stats
    winPercentage = team['won'] / team['GP'] * 100
    winPercentage = (winPercentage - winPercentageMin) / (winPercentageMax - winPercentageMin) * 100

    homeWinPercentage = team['homeW'] / (team['homeW'] + team['homeL']) * 100
    homeWinPercentage = (homeWinPercentage - homeWinPercentageMin) / (homeWinPercentageMax - homeWinPercentageMin) * 100

    awayWinPercentage = team['awayW'] / (team['awayW'] + team['awayL']) * 100
    awayWinPercentage = (awayWinPercentage - awayWinPercentageMin) / (awayWinPercentageMax - awayWinPercentageMin) * 100

    conferenceWinPercentage = team['confW'] / (team['confW'] + team['confL']) * 100
    conferenceWinPercentage = (conferenceWinPercentage - conferenceWinPercentageMin) / (conferenceWinPercentageMax - conferenceWinPercentageMin) * 100

    attendance = (team['attend'] - attendanceMin) / (attendanceMax - attendanceMin) * 100

    relevantFeatures = [ offensiveFieldGoalPercentage, offensiveFreeThrowPercentage, offensiveThreePointPercentage, offensiveReboundsPerGame, offensiveAssistsPerGame, 
                        offensivePersonalFoulsPerGame, offensiveStealsPerGame, offensiveTurnoversPerGame, offensiveBlocksPerGame, offensivePointsPerGame, 
                        defensiveFieldGoalPercentage, defensiveFreeThrowPercentage, defensiveThreePointPercentage, defensiveReboundsPerGame, defensiveAssistsPerGame, 
                        defensivePersonalFoulsPerGame, defensiveStealsPerGame, defensiveTurnoversPerGame, defensiveBlocksPerGame, defensivePointsPerGame, winPercentage, 
                        homeWinPercentage, awayWinPercentage, conferenceWinPercentage, attendance ]
    
    powerRankingAcummulator = 0
    numberOfFeatures = len(relevantFeatures)
    averageFeatureWeight = 1 / numberOfFeatures

    for feature in relevantFeatures:
        powerRankingAcummulator += feature * averageFeatureWeight

    return powerRankingAcummulator

In [ ]:
for index, team in teams.iterrows(): 
    teams.at[index, 'powerRanking'] = team_power_ranking(team)

teams = teams.drop(columns=['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'won', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'attend'])
teams.to_csv('teams.csv', index=False)

### Coaches Rating

**Features**
- Win percentage
- Average Points Scored (Offensive Stats)
- Average Points Allowed (Defensive Stats)
- Player development? 
- Turnover Management
- Rebounding Performance
- Performance in Close Games

In [ ]:
# Win Percentage
winPercentageMax = 0  
winPercentageMin = 100

# Average Points Scored (Offensive Stats)
averagePointsScoredMax = 0 
averagePointsScoredMin = 999  

# Average Points Allowed (Defensive Stats)
averagePointsAllowedMax = 0
averagePointsAllowedMin = 999

# Player Development 
playerDevelopmentMax = 0  
playerDevelopmentMin = 100  

# Turnover Management
turnoverManagementMax = 0  
turnoverManagementMin = 999  

# Rebounding Performance
reboundingPerformanceMax = 0  
reboundingPerformanceMin = 999 

# Performance in Close Games
performanceInCloseGamesMax = 0
performanceInCloseGamesMin = 100 


In [ ]:
# merging coaches and teams table to have more statistics about the coaches
coaches_teams = pd.merge(coaches, teams, on=["year", "tmID"], how="inner")
coaches_teams.head()
coaches_teams.to_csv('coaches_teams.csv', index=False)

In [ ]:
# Iterate through the coaches_teams table
for index, row in coaches_teams.iterrows():
    # Calculate coach statistics

    winPercentage = (row['won_x'] / (row['won_x'] + row['lost_x'])) * 100
    if winPercentage > winPercentageMax:
        winPercentageMax = winPercentage
    if winPercentage < winPercentageMin:
        winPercentageMin = winPercentage

    averagePointsScored = row['o_pts'] / row['GP']
    if averagePointsScored > averagePointsScoredMax:
        averagePointsScoredMax = averagePointsScored
    if averagePointsScored < averagePointsScoredMin:
        averagePointsScoredMin = averagePointsScored

    averagePointsAllowed = row['d_pts'] / row['GP']
    if averagePointsAllowed > averagePointsAllowedMax:
        averagePointsAllowedMax = averagePointsAllowed
    if averagePointsAllowed < averagePointsAllowedMin:
        averagePointsAllowedMin = averagePointsAllowed

In [ ]:
# Print the calculated max and min values
print("Win Percentage Max: ", winPercentageMax)
print("Win Percentage Min: ", winPercentageMin)
print("Average Points Scored Max: ", averagePointsScoredMax)
print("Average Points Scored Min: ", averagePointsScoredMin)
print("Average Points Allowed Max: ", averagePointsAllowedMax)
print("Average Points Allowed Min: ", averagePointsAllowedMin)

In [ ]:
def coach_power_ranking(coach):
    # Calculate coach statistics using the provided columns

    winPercentage = (coach['won_x'] / (coach['won_x'] + coach['lost_x'])) * 100
    relativeScorePerformance = (coach['averagePointsScored'] - averagePointsScoredMin) / (averagePointsScoredMax - averagePointsScoredMin) * 100
    relativeDefensePerformance = (averagePointsAllowedMax - coach['averagePointsAllowed']) / (averagePointsAllowedMax - averagePointsAllowedMin) * 100

    

    relevantFeatures = [winPercentage, relativeScorePerformance, relativeDefensePerformance]

    powerRankingAccumulator = 0
    numberOfFeatures = len(relevantFeatures)
    averageFeatureWeight = 1 / numberOfFeatures

    for feature in relevantFeatures:
        powerRankingAccumulator += feature * averageFeatureWeight

    return powerRankingAccumulator

# Iterate through the coaches_teams table and calculate power rankings for coaches
for index, coach in coaches_teams.iterrows():
    coaches_teams.at[index, 'powerRanking'] = coach_power_ranking(coach)

# Drop the columns you no longer need
coaches_teams = coaches_teams.drop(columns=['won_x', 'lost_x', 'o_pts', 'd_pts'])

# Save the updated coaches_teams DataFrame to a CSV file
coaches_teams.to_csv('coaches_teams.csv', index=False)


## Merging datasets

### Merging Players with Players Teams dataset

This step was already done while calculating the player rankings. 

In [ ]:
#merge_players_players_teams = pd.merge(players, players_teams, how='inner', on='playerID')
#merge_players_players_teams.head()
#merge_players_players_teams.to_csv('players_players_teams.csv', index=False)

### Merging previous dataset with Teams dataset

In [ ]:
merge_teams_players_teams = pd.merge(merge_players_players_teams, teams, on=['tmID', 'year'] , how='inner')
#merge_teams_players_teams.to_csv('teams_players_teams.csv', index=False)
merge_teams_players_teams.head()

### Merging previous dataset with Coaches dataset

In [ ]:
merge_coaches_players_teams = pd.merge(merge_teams_players_teams, coaches, on=['tmID', 'year'], how='inner')
#merge_coaches_players_teams.to_csv('merge_coaches_players_teams.csv', index=False)
merge_coaches_players_teams.head()

### Merging previous dataset with Awards Players dataset

In [ ]:
merge_award_teams = pd.merge(merge_coaches_players_teams, awards_players, on=['playerID', 'year'], how='outer')
#merge_award_teams.to_csv('merge_award_teams_post.csv', index=False)
merge_award_teams.head()

### Merging previous dataset with Awards Coaches dataset

In [ ]:
final = pd.merge(merge_award_teams, awards_coaches, on=['coachID', 'year'], how='outer')
#final.to_csv('final.csv', index=False)
#final.drop('')
final.tail()

## Classification

Applying machine learning models.

target label: 'playoff'
to know if a team in a given year will make it to the playoffs

### Encoding

In [ ]:
from datetime import datetime

final['birthDate'].fillna(0, inplace=True)

# Define a function to calculate age from a date string
def calculate_age(date_str):
    if date_str == 0: return
    
    date_of_birth = datetime.strptime(str(date_str), "%Y-%m-%d")
    current_date = datetime.now()
    age_timedelta = current_date - date_of_birth
    age_years = age_timedelta.days // 365  # This is a simple approximation
    return int(age_years)

# Apply the function to the "date_of_birth" column and create a new "age" column
final['birthDate'] = final['birthDate'].apply(calculate_age)
final.rename(columns={'birthDate': 'age'}, inplace=True)

# Save the DataFrame back to a new CSV file with the "age" column
#final.to_csv("data_with_age.csv", index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the columns 'teamID', 'franchID', 'confID', 'name', 'arena'
#teams['college'] = label_encoder.fit_transform(teams['college'])
teams['name'] = label_encoder.fit_transform(teams['name'])
teams['arena'] = label_encoder.fit_transform(teams['arena'])

final.to_csv("final.csv", index=False)

### Models

#### Decision Tree Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Assuming df is your DataFrame containing the dataset
# Drop non-essential columns, such as player ID
final = final.drop(columns=['playerID', 'coachID'])

# Split the data into features (X) and the target variable (y)
X = final.drop('playoff', axis=1)
y = final['playoff']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Decision Tree model
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)